# Econometría Aplicada II
## Tarea 1
Importar librerías

In [1]:
%%capture
# Clonar repo si estamos en colab
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/ArturoSbr/EmtrAp2-hw01
    # !pip install scipy==1.7.3
    %cd EmtrAp2-hw01/cod

# Libs
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.api import OLS
from matplotlib import pyplot as plt

Importar datos

In [2]:
d1 = pd.read_csv('../dat/baseline.csv')
d2 = pd.read_csv('../dat/endline.csv')
d3 = pd.read_csv('../dat/completa.csv')

### 1. Balance
Tabla de balance por grupo de acuerdo a `T_nap`

In [3]:
# Seleccionar 10 variables basales
X = ['time_in_office','age_','female_','education_','sleep_night','no_of_children_','act_inbed',
     'an_12_number_of_awakenings','an_13_average_awakening_length','unemployed']

# Inicializar lista
d = []

# Medias de variables basales
for x in X:
    # Grupos
    b, a = d1.groupby('T_nap')[x].apply(np.array)
    # t-test
    test = stats.ttest_ind(a=a, b=b, equal_var=False, nan_policy='omit')
    # Agregar a lista
    d.append([x] + list(test))

# A tabla
t = pd.DataFrame(data=d, columns=['var','t','p']).sort_values('var')
t.round(3)

,var,t,p
6,act_inbed,1.255,0.210
1,age_,-0.249,0.804
7,an_12_number_of_awakenings,0.780,0.436
8,an_13_average_awakening_length,0.111,0.912
3,education_,-1.296,0.196
2,female_,-0.034,0.973
5,no_of_children_,0.533,0.594
4,sleep_night,0.612,0.541
0,time_in_office,-0.246,0.806
9,unemployed,0.491,0.624


Evaluación conjunta de significancia

In [4]:
# T_nap en función de controles
m = OLS(endog=d1['T_nap'], exog=d1[X].assign(const = 1)).fit()

# p-value de prueba
m.f_pvalue

0.8817119192790587

### 2. Efectos de tratamiento
Declarar todas las variables dependientes

#### a) Estimadores de Neyman

In [5]:
# Función para estimador de Neyman
def neyman(frame, treatment_col, values_col):
    # Sacar arreglos C y T
    m = frame[[treatment_col,values_col]].notna().all(axis=1)
    a, b = frame[m].groupby(treatment_col)[values_col].apply(np.array)
    # Estadístico t
    tau = np.mean(b) - np.mean(a)
    t = tau / np.sqrt(np.var(a, ddof=1) / len(a) + np.var(b, ddof=1) / len(b))
    # p-value
    p = 2 * (1 - stats.norm().cdf(np.abs(t)))
    return [values_col, tau, t, p]

# Diferencia de Neyman
d = [neyman(d2, 'T_nap', 'productivity')[1:]]
print(f'ATE sobre productividad: {d[0][0]}')

ATE sobre productividad: -170.53711387267913


#### b) Estimadores OLS sin controles

In [6]:
m = OLS(endog=d2['productivity'], exog=d2.assign(const = 1)[['const','T_nap']], missing='drop')
m = m.fit(cov_type='HC0')

# Agregar a `d`
d.append([m.params['T_nap'], m.tvalues['T_nap'], m.pvalues['T_nap']])
print(f'ATE sobre productividad: {d[1][0]}')

ATE sobre productividad: -170.53711387267884


#### c) Estimadores con controles
De acuerdo al paper, $X_i$ contiene `age_` en cuartiles, `female_` y la variable que indica si $i$ fue asignado a trabajar o a tomarse un break en vez de tomar una siesta.

Como esta pregunta usa la base con promedios durante los 20 días de estudio, la variable que indica la actividad asignada cada día a los individuos del grupo de control no está disponible.

In [7]:
# Edad a cuartiles y luego a dummies
d2['age_q'] = pd.qcut(x=d2['age_'], q=4, labels=[f'q{i}' for i in range(1,5)])
d2 = pd.get_dummies(data=d2, prefix='age_', prefix_sep='', columns=['age_q'], )

# Tratamiento y controles
X = ['T_nap','const','age_q2','age_q3','age_q4','female_']

# Correr regresión
m = OLS(endog=d2['productivity'], exog=d2.assign(const = 1)[X], missing='drop')
m = m.fit(cov_type='HC0')

# Agregar a `d`
d.append([m.params['T_nap'], m.tvalues['T_nap'], m.pvalues['T_nap']])
print(f'ATE sobre productividad: {d[2][0]}')

ATE sobre productividad: -212.86371743292182


#### d) Resultados a tabla

In [8]:
# Concatenar resultados
t = pd.DataFrame(data=d, columns=['tau','t','p'], index=['Neyman','OLS simple','OLS controles'])
t

,tau,t,p
Neyman,-170.537114,-0.958923,0.337598
OLS simple,-170.537114,-0.961244,0.336429
OLS controles,-212.863717,-1.263394,0.206447


#### e) Nuevas variables dependientes

In [9]:
# Crear índice de habilidades cognitivas
d2['cog'] = d2[['corsi_measure','hf_measure','pvt_measure']] \
    .apply(lambda x: (x - x.mean()) / x.std()).mean(axis=1)

# Nuevas variables dependientes
Y = ['nap_time_mins','sleep_report','happy','cog','typing_time_hr']

# Correr regresiones
d = []
for y in Y:
    m = OLS(endog=d2[y], exog=d2.assign(const = 1)[X], missing='drop')
    m = m.fit(cov_type='HC0')
    d.append([m.params['T_nap'], m.tvalues['T_nap'], m.pvalues['T_nap']])

# Resultados a tabla
t = pd.DataFrame(data=d, columns=['tau','t','p'], index=Y)

# Visualizar
t.transpose().round(3)

,nap_time_mins,sleep_report,happy,cog,typing_time_hr
tau,11.745,0.049,0.050,0.029,-0.018
t,39.444,1.119,1.422,0.531,-0.167
p,0.000,0.263,0.155,0.596,0.868


De acuerdo al modelo $\bar{y}_i = \beta T_i + X_i^T\gamma$, donde $X_i$ controla por la edad (en cuartiles) y por sexo, el efecto de estimado de tratamiento de tomar una siesta es:
1. Aumentar el promedio de minutos dormidos durante la siesta en de 0 a 11.6 minutos.
1. Aumentar el promedio de número de horas de sueño reportadas en 0.05 horas por día (pero no tiene significancia estadística).
1. Aumentar el promedio de la calificación de felicidad reportada en 0.03 puntos (pero no tiene significancia estadística)
1. Aumentar el índice promedio de desempeño cognitivo en 0.03 desviaciones estándar (pero no tiene significancia estadística)
1. Reducir el promedio de horas trabajadas en 0.083 unidades diarias (pero no tiene significancia estadística)

### 3. Fischer's Exact Test
#### a) El tratamiento no tiene efecto

In [10]:
fet = stats.permutation_test(data=(d2.loc[d2['T_nap'].eq(1) & d2['productivity'].notna(), 'productivity'],
                                   d2.loc[d2['T_nap'].eq(0) & d2['productivity'].notna(), 'productivity']),
                             statistic=lambda x, y: np.mean(x) - np.mean(y),
                             n_resamples=1000,
                             random_state=42)

print('p-value:', round(fet.pvalue, 3))

p-value: 0.308


#### b) Conclusión
El p-value de la diferencia de Neyman y de OLS sin controles es 0.329, mientras que con la falsificación de Fischer el p-value es 0.328. Estos p-values son muy parecidos entre sí, por lo que podemos concluir con un alto grado de certeza que el efecto de las siestas sobre la productividad no es estadísticamente significativo.

Cuando agregamos controles al modelo OLS, el p-value baja a 0.238, lo cual es menor a lo que solía ser, pero permance sin significancia estadística.

En resumen, todos los casos indican que las siestas no tienen un efecto estadísticamente significativo sobre la productividad de las personas.

### 4. Estratificación

In [11]:
# Crear casos con datos basales
d1[['aboveEarn','aboveSleep']] = d1[['earnings','sleep_night']].apply(lambda x: (x > x.median()).astype(int), axis=0)

# Agregar casos a `d2`
d2 = d2.merge(d1[['pid','aboveEarn','aboveSleep']], on='pid')

#### a) Número de observaciones asignadas a tratamiento en cada estrato

In [12]:
d1.groupby(['aboveEarn','aboveSleep'])['T_nap'].sum()

aboveEarn  aboveSleep
0          0             57
           1             45
1          0             45
           1             58
Name: T_nap, dtype: int64

In [13]:
d2.groupby(['aboveEarn','aboveSleep'])['T_nap'].sum()

aboveEarn  aboveSleep
0          0             57
           1             45
1          0             45
           1             58
Name: T_nap, dtype: int64

El número de observaciones asignadas a tratamiento en cada estrato suma a 122. Es decir, el 50% de la población fue asignada a tratamiento. Sin embargo, el número de personas en cada grupo no es constante, sino que a veces está por encima de este número (59 y 60) y a veces está por debajo (54 y 53).

#### b) Efectos por estrato y agregados
Efectos por estrato

In [14]:
# Inicializar lista
d = []

# Efecto por esstrato a cada variable
for y in Y:
    for e, s in [(0,0),(0,1),(1,0),(1,1)]:
        # Máscara
        m = d2['aboveEarn'].eq(e) & d2['aboveSleep'].eq(s)
        d.append([e, s, m.sum(), m.sum() / len(d2)] + neyman(d2[m], 'T_nap', y))

t = pd.DataFrame(data=d, columns=['aboveEarn','aboveSleep','ng','wg','depvar','tau','t','p'])
t = t.set_index(['depvar','aboveEarn','aboveSleep'])

# Visualizar resultados
t[['tau']].round(3)

tau
depvar         aboveEarn aboveSleep        
nap_time_mins  0         0           10.884
                         1           13.413
               1         0           11.479
                         1           11.395
sleep_report   0         0            0.153
                         1            0.010
               1         0            0.029
                         1           -0.015
happy          0         0           -0.032
                         1            0.047
               1         0            0.130
                         1            0.073
cog            0         0            0.218
                         1            0.054
               1         0           -0.142
                         1           -0.039
typing_time_hr 0         0           -0.153
                         1            0.197
               1         0           -0.051
                         1           -0.030

Efectos agregados

#### c) Efectos estratificados con OLS

In [15]:
# Declarar columnas
d2 = d2.assign(const = 1,
               aboveEarnT = d2['aboveEarn'].multiply(d2['T_nap']),
               aboveSleepT = d2['aboveSleep'].multiply(d2['T_nap']))

# Declarar variables independientes
X = ['const','T_nap','aboveEarn','aboveEarnT','aboveSleep','aboveSleepT']

# OLS para cada variable
d = []
for y in Y:
    m = d2[X + [y]].notna().all(axis=1)
    m = OLS(endog=d2.loc[m, y], exog=d2.loc[m, X], missing='raise').fit(cov_type='HC0')
    d.append(pd.concat([m.params, m.tvalues, m.pvalues], axis=1).assign(depvar = y))

# A tabla
t = pd.concat(d, axis=0).reset_index()
t.columns = ['beta','value','t','p','depvar']
t = t.set_index(['depvar','beta'])
t[t.index.get_level_values(1).isin(['T_nap','aboveEarnT','aboveSleepT'])].round(3)

value       t      p
depvar         beta                              
nap_time_mins  T_nap        11.462  21.216  0.000
               aboveEarnT   -0.716  -1.221  0.222
               aboveSleepT   1.218   2.073  0.038
sleep_report   T_nap         0.131   1.115  0.265
               aboveEarnT   -0.074  -0.971  0.331
               aboveSleepT  -0.093  -1.226  0.220
happy          T_nap        -0.002  -0.031  0.975
               aboveEarnT    0.094   1.363  0.173
               aboveSleepT   0.011   0.166  0.868
cog            T_nap         0.158   1.648  0.099
               aboveEarnT   -0.226  -2.080  0.038
               aboveSleepT  -0.031  -0.284  0.776
typing_time_hr T_nap        -0.079  -0.450  0.653
               aboveEarnT   -0.063  -0.288  0.773
               aboveSleepT   0.185   0.845  0.398

### 5. Atrición
#### a) Reportar atrición

In [16]:
t = d2.groupby('T_nap')['drop_indicator'].agg(['size','sum'])
t['pct'] = t['sum'].div(t['size']) * 100
t.round(2)

,size,sum,pct
T_nap,,,
0,209,81,38.76
1,205,23,11.22


#### b) Nuevo balance
Validez interna

In [17]:
# Variables dependientes
X = ['time_in_office','age_','female_','education_','sleep_night','no_of_children_','act_inbed',
     'an_12_number_of_awakenings','an_13_average_awakening_length','unemployed']

# Tabla de balance
d = []
for x in X:
    b, a = d2[d2['drop_indicator'].eq(0)].groupby('T_nap')[x].apply(np.array)
    test = stats.ttest_ind(a=a, b=b, equal_var=False, nan_policy='omit')
    d.append([x] + list(test))

t = pd.DataFrame(data=d, columns=['variable','t-stat','p-value'])
t.sort_values('variable').round(3)

,variable,t-stat,p-value
6,act_inbed,-1.346,0.179
1,age_,0.455,0.650
7,an_12_number_of_awakenings,-0.597,0.551
8,an_13_average_awakening_length,-0.685,0.494
3,education_,-3.256,0.001
2,female_,0.577,0.564
5,no_of_children_,1.989,0.048
4,sleep_night,-0.502,0.616
0,time_in_office,-0.184,0.854
9,unemployed,0.864,0.388


In [18]:
m = OLS(endog=d2.loc[d2['drop_indicator'].eq(0), 'T_nap'],
        exog=d2.loc[d2['drop_indicator'].eq(0), X].assign(const = 1),
        missing='drop').fit(cov_type='HC0')
print(f'p-value de significancia conjunta: {round(m.f_pvalue, 3)}')

p-value de significancia conjunta: 0.095


Antes de la atrición, ninguna de las 10 variables tenía diferencias entre tratamiento y control que fueran estadísticamente significativas. Después de la atrición, los p-values son más chicos y algunas diferencias tienen significancia a nivel individual. Por ejemplo, las diferencias en `education_` y `no_of_children_` ahora son significativas al 1 y 5 porciento de confianza.

Al hacer la prueba de significancia conjunta usando el modelo $T_i = X_i^T \beta + U_i$, el p-value es aproximadamente 0.1. Es relativamente improbable que todos los coeficientes sean simultáneamente igual a cero, pero la significancia de la prueba cayó tras la atrición.

Representatividad externa

In [19]:
# Probar si diferencia es significativa Baseline VS Endline
d = []
for x in X:
    a, b = d2[x], d2.loc[d2['drop_indicator'].eq(0), x]
    test = stats.ttest_ind(a=a, b=b, equal_var=False, nan_policy='omit')
    d.append([x] + list(test))

# Resultados a tabla
t = pd.DataFrame(data=d, columns=['variable','t','p'])
t.sort_values('variable').round(3)

,variable,t,p
6,act_inbed,-0.221,0.825
1,age_,-0.718,0.473
7,an_12_number_of_awakenings,0.010,0.992
8,an_13_average_awakening_length,-0.117,0.907
3,education_,-0.722,0.470
2,female_,1.556,0.120
5,no_of_children_,1.403,0.161
4,sleep_night,0.005,0.996
0,time_in_office,-0.842,0.400
9,unemployed,0.743,0.458


La atrición parece no haber afectado la representatividad externa de la muestra. Ninguna de las diferencias son significativas individualmente.

#### c) Conclusión
La atrición fue sistemática entre el grupo de tratamiento y de control. Es decir, parece que el nivel de educación y el número de hijos determinan si alguien abandona o no el experimento. Esto nos lleva a un problema de validez interna porque los grupos de tratamiento y control después de la atrición no están balanceados.

Sin embargo, parece que la atrición no afectó la validez externa de la muestra, pues parece que las personas que abandonaron el estudio no afectaron las distribuciones de las variables de control. Ninguna de las 10 variables muestra una diferencia significativa antes y después de la atrición.

### 5. Lee Bounds
#### a) Perfiles
- Always Respondents: $S_i$ = 1 sin importar $T_i$
- Never Respondents: $S_i$ = 0 sin importar $T_i$
- Selective Respondents: $T_i = 0 \implies S_i = 0$, $T_i = 1 \implies S_i = 1$
- Counter-Selective Respondents: $T_i = 0 \implies S_i = 1$, $T_i = 1 \implies S_i = 0$

El supuesto de monotonicidad es que no existe alguno de los dos grupos de respuesta selectiva. En el contexto de este experimento, tiene sentido asumir que los Counter-Selective Respondents no existen porque el tratamiento es algo *bueno*. Es decir, si a alguien le toca tomar una siesta durante sus horas de trabajo, es razonable pensar que el tratamiento es algo deseable y por ende no incentivaría a los individuos a abandonar el experimento.

In [20]:
# Columna S_i
d2['S'] = 1 - d2['drop_indicator']

# Casos
t = d2.groupby(['T_nap','S']).size()
t.unstack().transpose()

T_nap,0,1
S,,
0,81,23
1,128,182


In [21]:
# P(AR|T=0)
par = 128 / (81 + 128)

# P(SR|T=1)
psr = 182 / (182 + 23) - par

# Probabilidades
print(f'P(AR) = {par}', f'P(SR) = {psr}', f'P(NR) = {1 - (par + psr)}', sep='\n')

P(AR) = 0.6124401913875598
P(SR) = 0.2753646866612207
P(NR) = 0.1121951219512195


#### b) Lee Bounds

In [22]:
# Arreglos de S_i = 1
a, b = d2[d2['S'].eq(1)].groupby('T_nap')['productivity'].apply(np.array)

# Lower bound
lb = b[b <= np.quantile(b, 1 - psr).mean()].mean() - a.mean()

# Upper bound
ub = b[b >= np.quantile(b, psr)].mean() - a.mean()

# Bounds
print(f'El ATE de los AR está en [{lb, ub}]')

El ATE de los AR está en [(-1779.6532923733384, -256.33863398462927)]


#### c) Comparación
Los resultados de la pregunta 2 no tienen por qué estar centrados en los Lee Bounds porque estiman el efecto para toda la población, mientras que el intervalo de esta pregunta acota el efecto de tratamiento para los Always Respondents.

Lo único que sí podemos ver es que el ATE observado en la muestra completa es mayor al ATE de los Always Respondents.

## 2. Matching